In [2]:
## Name: Chandni Patel
## ID: A20455322
## CS 512 - Fall 2020
## Extract Chessboard Features

import numpy as np
import cv2


In [3]:
class Extract_Features:
    
    #initialling the image processor
    def __init__(self, img):        
        self.H=7
        self.W=7
        self.TITLE = 'Extract Features'
        self.support_keys = {ord('h'),ord('i'),ord('w'),ord('f')}   
        self.img_h()                
        self.in_img = cv2.imread(img)    
        self.gray_img = cv2.cvtColor(self.in_img, cv2.COLOR_BGR2GRAY)
        self.curr_img = np.copy(self.in_img) 
        self.points_2D_chess = []
        self.points_2D_click = []        
        self.get_preview() 

    #showing the image and waiting for key press
    def get_preview(self):           
        cv2.imshow(self.TITLE, self.curr_img) 
        cv2.setMouseCallback(self.TITLE, self.clickevent) 
        while True:   
            k = cv2.waitKey(10) & 0xFF
            if k == 27:
                self.exit()
                break                
            if k in self.support_keys:
                self.command_key(k)               

    #controlling all support keys
    def command_key(self, k):        
        if k == ord('h'):
            self.img_h()     
        if k == ord('i'):
            self.img_i() 
        if k == ord('w'):
            self.img_w()            
        if k == ord('f'):
            self.points_2D_chess = []
            self.img_f()

    #exit
    def exit(self):
        cv2.destroyAllWindows()
    
    #display a short description of the program, command line argument and support support_keys
    def img_h(self):
        print("\nSupport support_keys are as below:")  
        print("Left mouse click -> Mark feature point")
        print("f -> Extract Chessboard feature points")
        print("h -> Display help")
        print("i -> Reload the original image") 
        print("w -> Save the current image and features extracted")
        print("ESC -> Exit\n")
        
    #reload the original images
    def img_i(self):        
        self.points_2D_chess = []
        self.points_2D_click = []
        self.curr_img = np.copy(self.in_img)
        cv2.imshow(self.TITLE, self.curr_img)  
        print('\n***** REFRESH *****\n')

    #save the current processed images
    def img_w(self):
        H = self.H
        W = self.W
        # 3D World points: (0,0,0), (1,0,0), (2,0,0), ... , (6,6,0)
        points_3D = np.zeros((H*W,3), np.float32)
        points_3D[:,:2] = np.mgrid[0:H,0:W].T.reshape(-1,2)
        for i in range(len(points_3D)):
            points_3D[i][0] *= 50 #50mm
            points_3D[i][1] *= 50 #50mm
            points_3D[i][2] = 1 #z=1
        
        if (len(self.points_2D_chess) == 1):
            points = self.points_2D_chess[0]
            f = open('points_3Dto2D.txt',"w+")
            for i in range(len(points)):
                X = str(points_3D[i][0])
                Y = str(points_3D[i][1])
                Z = str(points_3D[i][2])
                x = str(points[i][0][0])
                y = str(points[i][0][1])                
                line = X + " " + Y + " " + Z + " " + x + " " + y
                if (i != 0):
                    line = "\n" + line
                f.write(line)
            f.close() 
        
        if (len(self.points_2D_click) > 0):
            points = self.points_2D_click
            f = open('marked_2D.txt',"w+")
            for i in range(len(points)):
                x = str(points[i][0])
                y = str(points[i][1])
                line = x + " " + y
                if (i != 0):
                    line = "\n" + line
                f.write(line)
            f.close() 
        
        cv2.imwrite('out.jpg', self.curr_img)
        print('\n***** SAVED *****\n')
        
    def img_f(self):
        # Find the chess board corners
        H = self.H
        W = self.W
        patternfound, corners = cv2.findChessboardCorners(self.gray_img, (H,W), None)        
        if patternfound == True:
            final_corners = cv2.cornerSubPix(self.gray_img, corners, (11,11), (-1,-1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            self.points_2D_chess.append(final_corners)  
            self.curr_img = cv2.drawChessboardCorners(self.curr_img, (H,W), final_corners, patternfound)
            cv2.imshow(self.TITLE, self.curr_img)             
            print(len(self.points_2D_chess[0]))
        else:
            print(0)
        
    def clickevent(self, event, x, y, flags, params):
        # on left mouse click
        if event == cv2.EVENT_LBUTTONDOWN: 
            print(x, ' ', y)
            self.points_2D_click.append((x,y)) 
            self.curr_img = cv2.circle(self.curr_img, (x,y), radius = 10, color = (0, 255, 0), thickness = 2)
            cv2.imshow(self.TITLE, self.curr_img)
        

In [4]:
x = Extract_Features('chessboard.jpg')


Support support_keys are as below:
Left mouse click -> Mark feature point
f -> Extract Chessboard feature points
h -> Display help
i -> Reload the original image
w -> Save the current image and features extracted
ESC -> Exit

49
54   309
68   265
80   224
93   181
453   213
460   172
466   133

***** SAVED *****

